In [ ]:
from IMP_CW_env import MESCEnv
import numpy as np 
from scipy.optimize import minimize, basinhopping
import torch
import timer

from HeuristicPolicy_file import HeuristicPolicy, Optimizer
from PPO_file import PPO
from utils import plot_reward_evolution, read_log_file, plot_reward_distribution

%matplotlib inline
# %matplotlib ipympl
import matplotlib.pyplot as plt

# **Alternative environment**
### **Definition**
Instantiate environment used for testing purposes.

In [ ]:
n_retailers = [2]
n_DCs = [1]
n_suppliers = 1
supply_chain_structure = [n_retailers , n_DCs, n_suppliers]

demand_dist_param = [{'mu': 5}, # Mo-Thu
                     {'mu': 7}, # Fri
                     {'mu': 20}] # Sat-Sun

env_test = MESCEnv(supply_chain_structure, demand_dist_param=demand_dist_param)

Customize features of DCs and retailers.

In [ ]:
# Stress test capacity and lead times at distribution center
env_test.DCs[0].capacity = env_test.DCs[0].capacity/2
env_test.DCs[0].capacity_violation_cost += 20
env_test.DCs[0].lead_time += 1 
env_test.DCs[0].holding_cost = env_test.DCs[0].holding_cost * 2

# Retailer 0: premium customer 
env_test.retailers[0].lead_time = 1
env_test.retailers[0].lost_sales_cost = env_test.retailers[0].lost_sales_cost * 2 
env_test.retailers[0].capacity_violation_cost = env_test.retailers[0].capacity_violation_cost * 2

# Retailer 2: higher rotating inventory
env_test.retailers[0].capacity -= 25
env_test.retailers[1].fixed_order_cost = env_test.retailers[1].fixed_order_cost / 2

# Sold product is more expensive
env_test.retailers[0].unit_price += 10
env_test.retailers[1].unit_price += 10

# **Training phase** 
Find optimal policy using students' PPO implementation.
<!-- <span style="color:red">**TO DO**</span> -->

In [ ]:
# https://medium.com/data-science/limiting-a-python-functions-execution-time-using-a-decorator-and-multiprocessing-6fcfe01da6f8